In [187]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from statsmodels.stats.multitest import multipletests

In [188]:
frac = pd.read_csv("../../../3_purity/output/epic/epic_c1.txt", index_col=0)
frac.index = frac.index.str.replace(".", "-", regex=False)
frac.head()

,Bcells,CAFs,CD4_Tcells,CD8_Tcells,Endothelial,Macrophages,NKcells,otherCells,cluster
sampleID,,,,,,,,,
TCGA-AZ-6600-01A,0.1591,0.0332,0.2615,0.0120,0.4777,0.0189,0.0000,0.0375,1
TCGA-A6-5662-01A,0.0000,0.0035,0.0000,0.0000,0.5149,0.0000,0.0002,0.4813,1
TCGA-F4-6809-01A,0.1412,0.0119,0.1818,0.1690,0.4503,0.0065,0.0000,0.0393,1
TCGA-G4-6315-01A,0.0346,0.0584,0.3553,0.5103,0.0000,0.0414,0.0000,0.0000,1
TCGA-G4-6303-01A,0.0000,0.0130,0.0738,0.1511,0.2382,0.0114,0.0001,0.5125,1


In [189]:
cd8 = frac["CD8_Tcells"]
cd4 = frac["CD4_Tcells"]
#purity = frac["otherCells"]
#bcells = frac["Bcells"]
#cafs = frac["CAFs"]

In [191]:
expr = pd.read_csv("../../output/nr_gx.csv", index_col=0)
expr.head()

,ENSG00000003137,ENSG00000003989,ENSG00000004468,ENSG00000004776,ENSG00000004897,ENSG00000005020,ENSG00000005108,ENSG00000005175,ENSG00000005249,ENSG00000005513,...,ENSG00000275832,ENSG00000276070,ENSG00000276409,ENSG00000276418,ENSG00000277161,ENSG00000277957,ENSG00000278318,ENSG00000278615,ENSG00000280987,cl
Ensembl_ID,,,,,,,,,,,,,,,,,,,,,
TCGA-AA-3870-01A,-1.286937,-0.705065,-0.929976,0.421267,-0.997507,-0.265709,-0.103897,-1.165818,-1.377329,-1.587388,...,-0.253259,0.266248,-1.450667,-1.611563,-1.161838,-2.334001,-0.188997,-1.147065,-1.862966,1
TCGA-AA-3696-01A,-3.366378,-1.159320,-3.058207,-4.132610,-2.334947,-1.981988,-2.588803,-1.703634,-2.621653,-1.268934,...,-2.634787,-4.024211,-3.242426,-0.918416,-3.214611,-2.334001,-2.386222,-2.687510,-2.689645,1
TCGA-AA-3680-01A,-1.808234,-1.757157,-0.697353,-0.997116,-0.699058,0.118828,-0.756222,-0.650046,-0.791427,-1.027772,...,-1.111063,-2.078301,-1.296516,-0.512951,-0.695219,-2.334001,-0.188997,-1.793692,-1.544512,1
TCGA-A6-2682-01A,-2.806763,-0.840867,-1.805445,-1.041568,-1.856602,-1.196528,-2.029188,-1.708363,-2.459134,-2.973682,...,-1.839858,-0.274707,-1.632988,-2.304710,-1.594702,-2.334001,-2.386222,-1.517438,-3.942408,1
TCGA-AA-A02H-01A,-2.673231,-2.140150,-2.721735,-2.398009,-2.097473,-2.276553,-2.183338,-1.766916,-2.564495,-2.057392,...,-1.501084,-2.152409,-1.296516,-2.304710,-2.330409,-2.334001,-2.386222,-1.588897,-3.536943,1


In [192]:
if set(frac.index).issubset(set(expr.columns)):
    expr_sxg = expr.T
else:
    expr_sxg = expr  # already samples x genes

In [193]:
common = expr_sxg.index.intersection(frac.index)
expr_sxg = expr_sxg.loc[common]
cd8 = cd8.loc[common]
cd4 = cd4.loc[common]

In [194]:
onco_tsg = pd.read_csv("../../../1_deg/output/degs_tsg_onco.csv", index_col=0)
oncogenes = onco_tsg[onco_tsg["is_onco"]==True].index.to_list()
tsgs = onco_tsg[onco_tsg["is_tsg"]==True].index.to_list()

onco_tsg.head()

,logFC,adj.P.Val,hgnc_symbol,is_tsg,is_onco
Ensembl_ID,,,,,
ENSG00000163629,2.014847,2.085728e-06,PTPN13,True,False
ENSG00000023445,1.957911,2.964434e-07,BIRC3,True,True
ENSG00000134982,1.934692,1.799402e-10,APC,True,False
ENSG00000126778,1.887040,2.768037e-04,SIX1,False,True
ENSG00000119508,1.853774,2.032135e-05,NR4A3,False,True


In [195]:
genes = [g for g in (oncogenes + tsgs) if g in expr_sxg.columns]
print("Genes found:", len(genes), "Missing:", len(set(oncogenes+tsgs) - set(genes)))

def corr_table(cell_series, cell_name):
    rows = []
    for g in genes:
        x = cell_series.values
        y = expr_sxg[g].values
        # drop NaNs pairwise
        mask = np.isfinite(x) & np.isfinite(y)
        rho, p = spearmanr(x[mask], y[mask])
        onco = bool(onco_tsg.loc[g, "is_onco"])
        tsg  = bool(onco_tsg.loc[g, "is_tsg"])
        gene_type = "Oncogene/TSG" if (onco and tsg) else ("Oncogene" if onco else ("TSG" if tsg else "Other"))

        rows.append((g, gene_type, cell_name, rho, p))
    out = pd.DataFrame(rows, columns=["gene", "type", "cell", "spearman_rho", "pval"])
    out["qval_fdr"] = multipletests(out["pval"], method="fdr_bh")[1]
    return out.sort_values(["qval_fdr", "pval"])

res_cd8 = corr_table(cd8, "CD8")
res_cd4 = corr_table(cd4, "CD4")

res = pd.concat([res_cd8, res_cd4], ignore_index=True)
res = res.drop_duplicates()
res.to_csv("CD4_CD8_gene_correlations.csv", index=False)
res.head(14)


Genes found: 91 Missing: 0


,gene,type,cell,spearman_rho,pval,qval_fdr
0,ENSG00000214827,Oncogene,CD8,-0.525710,0.000588,0.038388
1,ENSG00000178053,TSG,CD8,-0.500671,0.001170,0.038388
2,ENSG00000134899,TSG,CD8,-0.497677,0.001266,0.038388
3,ENSG00000179094,TSG,CD8,-0.457757,0.003388,0.041109
4,ENSG00000113522,TSG,CD8,-0.457347,0.003420,0.041109
5,ENSG00000133703,Oncogene,CD8,-0.452025,0.003866,0.041109
6,ENSG00000073584,TSG,CD8,-0.451615,0.003902,0.041109
7,ENSG00000204370,TSG,CD8,-0.437285,0.005375,0.041109
8,ENSG00000126778,Oncogene,CD8,-0.429115,0.006412,0.041109
9,ENSG00000085224,TSG,CD8,-0.429096,0.006415,0.041109


In [196]:
from scipy.stats import zscore

def gene_set_score(df_sxg, gene_list):
    genes_present = [g for g in gene_list if g in df_sxg.columns]
    Z = df_sxg[genes_present].apply(zscore, axis=0, nan_policy="omit")
    return Z.mean(axis=1)

onc_score = gene_set_score(expr_sxg, oncogenes)
tsg_score = gene_set_score(expr_sxg, tsgs)

rho_cd8_onc, p_cd8_onc = spearmanr(cd8, onc_score, nan_policy="omit")
rho_cd8_tsg, p_cd8_tsg = spearmanr(cd8, tsg_score, nan_policy="omit")
rho_cd4_onc, p_cd4_onc = spearmanr(cd4, onc_score, nan_policy="omit")
rho_cd4_tsg, p_cd4_tsg = spearmanr(cd4, tsg_score, nan_policy="omit")

print("CD8 vs oncogene score:", rho_cd8_onc, p_cd8_onc)
print("CD8 vs TSG score:",     rho_cd8_tsg, p_cd8_tsg)
print("CD4 vs oncogene score:", rho_cd4_onc, p_cd4_onc)
print("CD4 vs TSG score:",     rho_cd4_tsg, p_cd4_tsg)


CD8 vs oncogene score: -0.34086090913799294 0.03371438246599937
CD8 vs TSG score: -0.36972660774967886 0.02052649569211863
CD4 vs oncogene score: -0.31219993179305655 0.05299941003114049
CD4 vs TSG score: -0.3494592023414738 0.029210232464502824
